In [1]:
from nsai_experiments.zoning_game.zl_gym import ZoningLangEnv

In [2]:
myenv = ZoningLangEnv(eval_num_procs=None)

In [3]:
myenv.grammar

<Grammar with 36 productions>

In [4]:
myenv.num_tokens

43

In [5]:
obs, *_ = myenv.reset(seed=47)
obs[:10]

[31, 0, 0, 0, 0, 0, 0, 0, 0, 0]

In [6]:
print(myenv.render().read())

Current program:
'''
Policy
'''
terminated = False, truncated = False.



In [7]:
obs, *_ = myenv.step((0, 0))
print(myenv.render().read())

Current program:
'''
Rule ;
Policy
'''
terminated = False, truncated = False.



In [8]:
obs[:10]

[41, 15, 31, 0, 0, 0, 0, 0, 0, 0]

In [9]:
myenv.current_program[0]

Rule

In [10]:
print("obs:", obs[:10])
print("nonterminals:", myenv.nonterminals)
print("Indices:", myenv.get_indices_of_nonterminals(obs))

obs: [41, 15, 31, 0, 0, 0, 0, 0, 0, 0]
nonterminals: {Policy, Constraint, Object, SubjectTile, DistanceConstraint, ObjectTile, Number, Subject, ClusterCountConstraint, Location, Rule, ClusterSizeConstraint}
Indices: [0, 2]


In [11]:
obs, *_ = myenv.step((0, 0), on_invalid="warn")
print(myenv.render().read())

Current program:
'''
Rule ;
Policy
'''
terminated = False, truncated = False.



Let's try a simple random policy:

In [12]:
import random
import numpy as np
random.seed(42)
np.random.seed(48)

# Reset the environment
obs, *_ = myenv.reset(seed=47)

# Iteratively apply random productions until terminated or truncated
terminated, truncated = False, False
while not (terminated or truncated):
    # Get indices of nonterminals in the current observation
    nonterminal_indices = myenv.get_indices_of_nonterminals(obs)
    
    if not nonterminal_indices:
        break  # No more nonterminals to expand
    
    # Select a random nonterminal index
    token_i = random.choice(nonterminal_indices)

    # # Get the nonterminal symbol at that index
    # nonterminal_symbol = myenv.int_to_symbol[obs[token_i]]
    
    # # Find all productions that match this nonterminal (LHS matches)
    # matching_productions = [(i, prod) for i, prod in enumerate(myenv.grammar.productions()) 
    #                        if prod.lhs() == nonterminal_symbol]
    
    # Get the nonterminal integer at that index
    nonterminal_int = obs[token_i]
    
    # Get all productions that match this nonterminal
    matching_productions = myenv.get_productions_for_nonterminal(nonterminal_int)
    
    # Extract indices and probabilities
    production_indices = [i for i, _ in matching_productions]
    probabilities = [prod.prob() for _, prod in matching_productions]
    
    # Select a random production weighted by probabilities
    production_i = random.choices(production_indices, weights=probabilities)[0]
    
    # Apply the action
    obs, reward, terminated, truncated, info = myenv.step((token_i, production_i))

# Print final result
print(myenv.render().read())
print(f"\nReward: {reward}")

Finished with reward 47.84000015258789
Current program:
'''
RESIDENTIAL must ( be_within 4 tiles_of BOARD_EDGE and ( ( form_cluster_with_fewer_than 5 tiles or ( not be_within 3 tiles_of RESIDENTIAL ) ) and form_cluster_with_fewer_than 4 tiles ) ) ;
DOWNTOWN must ( not ( be_within 4 tiles_of BOARD_HORIZONTAL_MEDIAN and form_fewer_than 4 separate_clusters ) ) ;
PARK must ( ( ( form_cluster_with_fewer_than 5 tiles and ( form_fewer_than 4 separate_clusters or ( ( ( ( not ( ( ( not be_within 2 tiles_of BOARD_HORIZONTAL_MEDIAN ) and ( not be_within 3 tiles_of DOWNTOWN ) ) or ( not ( not ( be_within 5 tiles_of BOARD_CORNER or ( not ( not be_within 5 tiles_of PARK ) ) ) ) ) ) ) or ( not ( form_cluster_with_fewer_than 4 tiles and ( ( form_cluster_with_fewer_than 5 tiles or be_within 3 tiles_of PARK ) and ( form_fewer_than 1 separate_clusters or form_cluster_with_fewer_than 5 tiles ) ) ) ) ) and ( not ( be_within 5 tiles_of BOARD_CORNER or form_fewer_than 2 separate_clusters ) ) ) or ( ( ( be_wi